In [13]:
from web3 import Web3
from web3.beacon import Beacon

In [14]:
web3 = Web3(Web3.HTTPProvider("https://mainnet.infura.io/v3/39e54952fe5548a5b4c69cc27528207f"))

In [15]:
block = web3.eth.get_block(14000000, full_transactions=False)

AttributeDict({'baseFeePerGas': 139541559304,
 'difficulty': 12316581093827601,
 'extraData': HexBytes('0x6175737472616c69612d736f75746865617374312d31'),
 'gasLimit': 30058561,
 'gasUsed': 8119826,
 'hash': HexBytes('0x9bff49171de27924fa958faf7b7ce605c1ff0fdee86f4c0c74239e6ae20d9446'),
 'logsBloom': HexBytes('0x083815041300081040f485088003de10046040491c3c0164189830ca3c2c0d1121040493008ba600924849820c8901130b0084038802610706000b6101b220d01210000400020b08480850880d0226a04024e107005114513410021080080451126063400a1084021c62bc08290008a0430081540c892545360c401410194021426103228249162800ac016601100d0520b96203432080086a00424d8214010b63180471318634d0ae00108041a1184488a10841a2610e0401e9001a504a4010098182020808155027009c02008e56070c28728114480490400801cac010290ae291a00ca0408b60204c128012601c100a24cc01e100897242832a825000c1e1'),
 'miner': '0xEA674fdDe714fd979de3EdF0F56AA9716B898ec8',
 'mixHash': HexBytes('0xa832679fcf3e71f0b29bd5913955151be5bf1fc59e135e3c7eb6e30f8442a5df'),
 'nonce': HexBytes('0x3

In [16]:
parent_hash = block.parentHash.hex()
uncles_hash = block.sha3Uncles.hex()
coinbase = block.miner  # lowercase
state_root = block.stateRoot.hex()
transactions_root = block.transactionsRoot.hex()
receipts_root = block.receiptsRoot.hex()
bloom = block.logsBloom.hex()
difficulty = block.difficulty
number = block.number
gas_limit = block.gasLimit
gas_used = block.gasUsed
timestamp = block.timestamp
extra_data = block.extraData.hex()
mix_hash = block.mixHash.hex()
nonce = block.nonce.hex()
base_fee_per_gas = block.baseFeePerGas  # Added for EIP-1559

In [17]:
difficulty_hex = hex(difficulty)
gas_limit_hex = hex(gas_limit)
gas_used_hex = hex(gas_used)
timestamp_hex = hex(timestamp)
number_hex = hex(number)
base_fee_per_gas = hex(base_fee_per_gas)

In [18]:
difficulty_hex = '0x' if difficulty_hex == '0x0' else difficulty_hex
gas_limit_hex = '0x' if gas_limit_hex == '0x0' else gas_limit_hex
gas_used_hex = '0x' if gas_used_hex == '0x0' else gas_used_hex
timestamp_hex = '0x' if timestamp_hex == '0x0' else timestamp_hex
# nonce = '0x' if nonce == '0x0000000000000000' else nonce
base_fee_per_gas = '0x' if base_fee_per_gas == '0x0' else base_fee_per_gas

In [19]:
coinbase_lower = coinbase.lower()
coinbase_lower

'0xea674fdde714fd979de3edf0f56aa9716b898ec8'

In [20]:
block_header = [
    parent_hash,  # parentHash
    uncles_hash,  # ommersHash
    coinbase_lower,  #beneficiary
    state_root,  # stateRoot
    transactions_root,  # transactionsRoot
    receipts_root,  # receiptsRoot
    bloom,  #logsBloom
    difficulty_hex,  # difficulty
    number_hex,  # number
    gas_limit_hex,  # gasLimit
    gas_used_hex,  # gasUsed
    timestamp_hex,  # timestamp
    extra_data,  # extraData
    mix_hash,  # mixHash
    nonce,  # nonce
    base_fee_per_gas,
]

In [21]:
for i in range(len(block_header)):
    # check if they are odd-sized, if so, add a leading 0
    if len(block_header[i]) % 2 != 0:
        block_header[i] = '0x0' + block_header[i][2:]
    block_header[i] = bytes.fromhex(block_header[i][2:])

In [22]:
import rlp

# rlp encode the values
rlp_encoded = rlp.encode(block_header)

'f9021ea00c9ef41f038aa58a4aa2810fda03d9d82aac9082c80283230fd74cb1cceb4b00a01dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d4934794ea674fdde714fd979de3edf0f56aa9716b898ec8a0fb3df73e7a41d500a374ee236dd613bb24f1a5fa5b80615a7953ea75c26c6a6da016b6ff83df3ef14f614c70ac29e8a05d102c6bed0e5882c284abf0120b89529ca0f7b9f8e92a4b420a27302ab91084e5499d3d711183a021f4956ff32de246e14eb90100083815041300081040f485088003de10046040491c3c0164189830ca3c2c0d1121040493008ba600924849820c8901130b0084038802610706000b6101b220d01210000400020b08480850880d0226a04024e107005114513410021080080451126063400a1084021c62bc08290008a0430081540c892545360c401410194021426103228249162800ac016601100d0520b96203432080086a00424d8214010b63180471318634d0ae00108041a1184488a10841a2610e0401e9001a504a4010098182020808155027009c02008e56070c28728114480490400801cac010290ae291a00ca0408b60204c128012601c100a24cc01e100897242832a825000c1e1872bc1dd80f1e41183d59f808401caa841837be6128461e0aeeb966175737472616c69612d736f75746865617374312d31a0a83

In [23]:
# hash the rlp encoded values
block_hash = web3.keccak(rlp_encoded).hex()
assert block_hash == block.hash.hex()

In [84]:
from mpt import MerklePatriciaTrie

storage = {}
trie = MerklePatriciaTrie(storage)

trie.update(b'0', block.transactions[0])
trie.update(b'1', block.transactions[1])

old_root = trie.root()
old_root_hash = trie.root_hash()

print("Root hash is {}".format(old_root_hash.hex()))

Root hash is 8bf2bf4c5224f2ca87b8846fcbf018f0d72afab1b03a246886c2862244e79961
